In [1]:
import opensim as osim
import numpy as np
import hyperobjects as ho
import common
import clifford as cf
from clifford import g3c
import clifford.tools.g3c as g3c_tools
from opensim import Vec3
from enum import Enum

c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\pyganja\__init__.py:2: UserWarning: Failed to import cef_gui, cef functions will be unavailable
  from .script_api import *
c:\Users\glaisne\mambaforge\envs\workspace\Lib\

In [2]:

G3c, blades_g3c, stuff = g3c.layout, g3c.blades, g3c.stuff

e1, e2, e3 = g3c.e1, g3c.e2, g3c.e3
e12, e13, e23 = g3c.e12, g3c.e13, g3c.e23
e31 = -e13
Px, Py, Pz = g3c.e23, -g3c.e13, g3c.e12
up, down, homo = g3c.up, g3c.down, g3c.homo
no, ni = g3c.eo, g3c.einf
E0 = g3c.E0
I_base = g3c.I_base

In [3]:
class MultiVectorType(Enum):
    NotBlade = -1
    Vector = 0
    EuclideanPoint = 1
    ConformalPoint = 2
    PointPair = 3
    Circle = 4
    Line = 5
    Sphere = 6
    Plane = 7

def blade_type(mv):
    if not isinstance(mv, cf.MultiVector):
        return MultiVectorType.NotBlade
    
    obj = g3c_tools.interpret_multivector_as_object(mv)
    if obj == -1:
        return MultiVectorType.NotBlade
    elif obj == 0:
        return MultiVectorType.Vector
    elif obj == 1:
        return MultiVectorType.EuclideanPoint
    elif obj == 2:
        return MultiVectorType.ConformalPoint
    elif obj == 3:
        return MultiVectorType.PointPair
    elif obj == 4:
        return MultiVectorType.Circle
    elif obj == 5:
        return MultiVectorType.Line
    elif obj == 6:
        return MultiVectorType.Sphere
    elif obj == 7:
        return MultiVectorType.Plane
    
def to_vector(v):
    if isinstance(v, osim.Vec3):
        v = v.to_numpy()
    return v[0]*e1 + v[1]*e2 + v[2]*e3

def to_point(p):
    return up(to_vector(p))

def to_array(mv):
    b_type = blade_type(mv)
    if b_type == MultiVectorType.Vector:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.EuclideanPoint:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.ConformalPoint:
        mv = down(mv)
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.Line:
        # return position then direction
        P, D = g3c_tools.line_to_point_and_direction(mv)
        if P == 0:
            return [np.array([0,0,0]), to_array(D)]
        return [to_array(P), to_array(D)]
    elif b_type == MultiVectorType.Plane:
        # return orthogonal proj of origin onto it THEN normal
        location = g3c_tools.get_nearest_plane_point(mv)
        normal = g3c_tools.get_plane_normal(mv)
        if location == 0:
            return [np.array([0,0,0]), to_array(normal)]
        return [to_array(location), to_array(normal)]
    else:
        raise TypeError(f"Can't convert {b_type} as numpy array.")

def rotation_axis_in_joint_frame(model, Q, last_frame):
    W_in_ground = common.frame_jacobian(model, Q, last_frame)
    W_in_joints = []
    all_joints = [j for j in model.getJointSet()]
    for i, w_in_ground in enumerate(W_in_ground.T):
        Ji = all_joints[i // 3]
        w_in_joint = model.getGround().expressVectorInAnotherFrame(Q, Vec3(*w_in_ground), Ji.getParentFrame()).to_numpy()
        W_in_joints.append(w_in_joint)
    return np.array(W_in_joints).T

def lever_arms(model, Q, W, origin, frame_origin, insertion, frame_insertion):
    if blade_type(origin) == MultiVectorType.NotBlade:
        if not isinstance(origin, osim.Vec3):
            origin = osim.Vec3(*origin)
    else:
        origin = osim.Vec3(*to_array(origin))

    if blade_type(insertion) == MultiVectorType.NotBlade:
        if not isinstance(insertion, osim.Vec3):
            insertion = osim.Vec3(*insertion)
    else:
        insertion = osim.Vec3(*to_array(insertion))

    lever_arms = []
    for i, c in enumerate(model.getCoordinateSet()):
        # c: osim.Coordinate = c
        FJ: osim.Joint = c.getJoint().getParentFrame()
        O_in_joint = frame_origin.findStationLocationInAnotherFrame(Q, origin, FJ).to_numpy()
        I_in_joint = frame_insertion.findStationLocationInAnotherFrame(Q, insertion, FJ).to_numpy()
        moment = np.cross(I_in_joint, O_in_joint) / np.linalg.norm(I_in_joint - O_in_joint)
        lever_arm = W[:, i] @ moment
        lever_arms.append(lever_arm)
    
    return np.array(lever_arms)

In [109]:
np.random.seed(42)

model = osim.Model()
ground: osim.Ground = model.getGround()

body1 = osim.Body("body1", 1.0, osim.Vec3(0), osim.Inertia(0))
body2 = osim.Body("body2", 1.0, osim.Vec3(0), osim.Inertia(0))
body3 = osim.Body("body3", 1.0, osim.Vec3(0), osim.Inertia(0))

# Joints are described such that their rotation/translation axes form the identity in the parent frame.
joint1 = osim.BallJoint("joint1", ground, Vec3(0), Vec3(0), body1, Vec3(0,0.5,0), Vec3(0))
joint2 = osim.BallJoint("joint2", body1, Vec3(0, -0.5, 0), Vec3(0), body2, Vec3(0,0.3,0), Vec3(0))

# Only translational joint
off_x = osim.Body("joint3_x_child", 1.0, osim.Vec3(0), osim.Inertia(0))
off_y = osim.Body("joint3_y_child", 1.0, osim.Vec3(0), osim.Inertia(0))
joint3 = osim.SliderJoint("joint3_x", body2, Vec3(0, -0.5, 0), Vec3(0), off_x, Vec3(0), Vec3(0))
joint3_y = osim.SliderJoint("joint3_y", off_x, Vec3(0), Vec3(0, 0, osim.SimTK_PI/2), off_y, Vec3(0), Vec3(0))
joint3_z = osim.SliderJoint("joint3_z", off_y, Vec3(0), Vec3(0, -osim.SimTK_PI/2, 0), body3, Vec3(0,0.06,0), Vec3(0))

# Cable spanning one rotational joint only
C1 = osim.PathActuator()
C1_body_origin = ground
C1_body_insertion = body1
C1_O = np.random.uniform(size=3)
C1_I = np.random.uniform(size=3)
C1.addNewPathPoint("C1_O", C1_body_origin, Vec3(*C1_O))
C1.addNewPathPoint("C1_I", C1_body_insertion, Vec3(*C1_I))

# Cable spanning two rotational joints
C2 = osim.PathActuator()
C2_body_origin = ground
C2_body_insertion = body2
C2_O = np.random.uniform(size=3)
C2_I = np.random.uniform(size=3)
C2.addNewPathPoint("C2_O", C2_body_origin, Vec3(*C2_O))
C2.addNewPathPoint("C2_I", C2_body_insertion, Vec3(*C2_I))

# Cable spanning one translation joint only
C3 = osim.PathActuator()
C3_body_origin = body2
C3_body_insertion = body3
C3_O = np.random.uniform(size=3)
C3_I = np.random.uniform(size=3)
C3.addNewPathPoint("C3_O", C3_body_origin, Vec3(*C3_O))
C3.addNewPathPoint("C3_I", C3_body_insertion, Vec3(*C3_I))

# Cable spanning one translation joint and one rotational joint
C4 = osim.PathActuator()
C4_body_origin = body1
C4_body_insertion = body3
C4_O = np.random.uniform(size=3)
C4_I = np.random.uniform(size=3)
C4.addNewPathPoint("C4_O", C4_body_origin, Vec3(*C4_O))
C4.addNewPathPoint("C4_I", C4_body_insertion, Vec3(*C4_I))

model.addBody(body1)
model.addBody(body2)
model.addBody(body3)
model.addBody(off_x)
model.addBody(off_y)

model.addJoint(joint1)
model.addJoint(joint2)
model.addJoint(joint3)
model.addJoint(joint3_y)
model.addJoint(joint3_z)

model.addForce(C1)
model.addForce(C2)
model.addForce(C3)
model.addForce(C4)
# model.addJoint(joint3)
Q_null = model.initSystem()
model.realizePosition(Q_null)
model.equilibrateMuscles(Q_null)

Q_val = np.random.uniform(-np.pi, np.pi, size=(4, 9))
all_states = [osim.State(Q_null) for _ in Q_val]

for state in all_states:
    model.realizePosition(state)
    model.equilibrateMuscles(state)

for k, state in enumerate(all_states):
    for i, c in enumerate(model.getCoordinateSet()):
        c.setValue(state, Q_val[k, i])

Q1 = all_states[0]
Q2 = all_states[1]
Q3 = all_states[2]
Q4 = all_states[3]

In [110]:
for c in model.getCoordinateSet():
    print(c.getName())

Jw_Q1 = common.frame_jacobian(model, all_states[3], body3)
Jl_Q1 = common.station_jacobian(model, all_states[3], Vec3(*[1,10,10]), body3)
print(Jw_Q1.round(2))
print(Jl_Q1.round(2))

W_joint1 = Jw_Q1[:,:3]
W_joint2 = Jw_Q1[:,3:6]
W_joint3 = Jl_Q1[:,6:]

print(ground.expressVectorInAnotherFrame(all_states[3], Vec3(*W_joint3[:,0]), joint3.getParentFrame()).to_numpy().round(3))
print(ground.expressVectorInAnotherFrame(all_states[3], Vec3(*W_joint3[:,1]), joint3.getParentFrame()).to_numpy().round(3))
print(ground.expressVectorInAnotherFrame(all_states[3], Vec3(*W_joint3[:,2]), joint3.getParentFrame()).to_numpy().round(3))

joint1_coord_0
joint1_coord_1
joint1_coord_2
joint2_coord_0
joint2_coord_1
joint2_coord_2
joint3_x_coord_0
joint3_y_coord_0
joint3_z_coord_0
[[ 1.    0.    0.    0.73  0.57  0.37  0.    0.    0.  ]
 [ 0.    1.    0.   -0.39 -0.1   0.92  0.    0.    0.  ]
 [ 0.    0.    1.    0.56 -0.82  0.15  0.    0.    0.  ]]
[[  0.   -11.75  -8.05   0.4    7.75 -12.69   0.18  -0.91   0.37]
 [ 11.75   0.     9.72  14.97  -1.23   6.17  -0.2   -0.4   -0.89]
 [  8.05  -9.72   0.     9.8    5.57  -6.48   0.96   0.09  -0.26]]
[ 1.  0. -0.]
[ 0.  1. -0.]
[-0. -0.  1.]


In [21]:
W = rotation_axis_in_joint_frame(model, Q1, B_insertion)
W.round(5)

array([[ 1., -0., -0.,  1., -0., -0.,  1.,  0.,  0.,  1.,  0., -0.],
       [-0.,  1.,  0., -0.,  1.,  0.,  0.,  1., -0.,  0.,  1., -0.],
       [-0.,  0.,  1., -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  1.]])

In [111]:
coordinates = model.getCoordinateSet()

print("CABLE 1:")
C1_LA_derivatives_Q1 = common.lever_arm_matrix(Q1, [C1], coordinates).ravel()
C1_LA_derivatives_Q2 = common.lever_arm_matrix(Q2, [C1], coordinates).ravel()
C1_LA_derivatives_Q3 = common.lever_arm_matrix(Q3, [C1], coordinates).ravel()
C1_LA_derivatives_Q4 = common.lever_arm_matrix(Q4, [C1], coordinates).ravel()
print("Q1:", C1_LA_derivatives_Q1.round(4).tolist())
print("Q2:", C1_LA_derivatives_Q2.round(4).tolist())
print("Q3:", C1_LA_derivatives_Q3.round(4).tolist())
print("Q4:", C1_LA_derivatives_Q4.round(4).tolist())

print("\nCABLE 2:")
C2_LA_derivatives_Q1 = common.lever_arm_matrix(Q1, [C2], coordinates).ravel()
C2_LA_derivatives_Q2 = common.lever_arm_matrix(Q2, [C2], coordinates).ravel()
C2_LA_derivatives_Q3 = common.lever_arm_matrix(Q3, [C2], coordinates).ravel()
C2_LA_derivatives_Q4 = common.lever_arm_matrix(Q4, [C2], coordinates).ravel()
print("Q1:", C2_LA_derivatives_Q1.round(4).tolist())
print("Q2:", C2_LA_derivatives_Q2.round(4).tolist())
print("Q3:", C2_LA_derivatives_Q3.round(4).tolist())
print("Q4:", C2_LA_derivatives_Q4.round(4).tolist())

print("\nCABLE 3:")
C3_LA_derivatives_Q1 = common.lever_arm_matrix(Q1, [C3], coordinates).ravel()
C3_LA_derivatives_Q2 = common.lever_arm_matrix(Q2, [C3], coordinates).ravel()
C3_LA_derivatives_Q3 = common.lever_arm_matrix(Q3, [C3], coordinates).ravel()
C3_LA_derivatives_Q4 = common.lever_arm_matrix(Q4, [C3], coordinates).ravel()
print("Q1:", C3_LA_derivatives_Q1.round(4).tolist())
print("Q2:", C3_LA_derivatives_Q2.round(4).tolist())
print("Q3:", C3_LA_derivatives_Q3.round(4).tolist())
print("Q4:", C3_LA_derivatives_Q4.round(4).tolist())

print("\nCABLE 4:")
C4_LA_derivatives_Q1 = common.lever_arm_matrix(Q1, [C4], coordinates).ravel()
C4_LA_derivatives_Q2 = common.lever_arm_matrix(Q2, [C4], coordinates).ravel()
C4_LA_derivatives_Q3 = common.lever_arm_matrix(Q3, [C4], coordinates).ravel()
C4_LA_derivatives_Q4 = common.lever_arm_matrix(Q4, [C4], coordinates).ravel()
print("Q1:", C4_LA_derivatives_Q1.round(4).tolist())
print("Q2:", C4_LA_derivatives_Q2.round(4).tolist())
print("Q3:", C4_LA_derivatives_Q3.round(4).tolist())
print("Q4:", C4_LA_derivatives_Q4.round(4).tolist())

CABLE 1:
Q1: [-0.6071, 0.0168, 0.2889, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Q2: [-0.4844, 0.1183, 0.0942, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Q3: [0.4914, -0.3004, 0.1387, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Q4: [-0.6391, 0.0188, 0.3025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

CABLE 2:
Q1: [-0.0757, -0.5911, 0.859, -0.0177, -0.3516, -0.0146, 0.0, 0.0, 0.0]
Q2: [0.2912, 0.4662, -0.7, 0.2468, 0.6069, -0.6187, 0.0, 0.0, 0.0]
Q3: [0.2049, -0.4535, 0.6336, -0.3018, 0.2304, 0.1083, 0.0, 0.0, 0.0]
Q4: [-0.8652, -0.0174, 0.1086, 0.2158, -0.5799, 0.2817, 0.0, 0.0, 0.0]

CABLE 3:
Q1: [0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0931, 0.7677, 0.634]
Q2: [0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.3731, 0.9277, 0.0074]
Q3: [0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.1766, 0.7252, -0.6655]
Q4: [-0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.576, 0.7896, 0.2113]

CABLE 4:
Q1: [-0.0, 0.0, 0.0, 0.6732, 0.2562, -0.8065, -0.18, 0.5883, 0.7884]
Q2: [-0.0, 0.0, -0.0, -0.3977, 0.7471, -0.6854, 0.1165, 0.9373, -0.3286]
Q3: [-0.0, 0.0, -0.0, -0.0698, 0.1566, -0.1533, -0.01

In [156]:
Oa = C3_body_origin.findStationLocationInAnotherFrame(Q1, Vec3(*C3_O), joint3.getParentFrame())
Ia = C3_body_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*C3_I), joint3.getParentFrame())

LL = to_point(Oa)^to_point(Ia)^ni
print(LL.normal())
p, d = g3c_tools.line_to_point_and_direction(LL.normal())
print(d)
print(C3_LA_derivatives_Q1.round(4)[6:])
D1 = C3_LA_derivatives_Q1[6:]
D2 = C3_LA_derivatives_Q2[6:]
D3 = C3_LA_derivatives_Q3[6:]
D4 = C3_LA_derivatives_Q4[6:]


-(0.57277^e124) - (0.57277^e125) - (0.51085^e134) - (0.51085^e135) - (0.09307^e145) - (0.31204^e234) - (0.31204^e235) - (0.76771^e245) - (0.63401^e345)
-(0.09307^e1) - (0.76771^e2) - (0.63401^e3)
[0.0931 0.7677 0.634 ]
[0.57602652 0.78963652 0.21134717]


In [112]:
C1_n_joint1_Q1 = C1_LA_derivatives_Q1[:3]
C1_n_joint1_Q2 = C1_LA_derivatives_Q2[:3]
C1_n_joint1_Q3 = C1_LA_derivatives_Q3[:3]
C1_n_joint1_Q4 = C1_LA_derivatives_Q4[:3]

C2_n_joint1_Q1 = C2_LA_derivatives_Q1[:3]
C2_n_joint1_Q2 = C2_LA_derivatives_Q2[:3]
C2_n_joint1_Q3 = C2_LA_derivatives_Q3[:3]
C2_n_joint1_Q4 = C2_LA_derivatives_Q4[:3]
C2_n_joint2_Q1 = C2_LA_derivatives_Q1[3:6]
C2_n_joint2_Q2 = C2_LA_derivatives_Q2[3:6]
C2_n_joint2_Q3 = C2_LA_derivatives_Q3[3:6]
C2_n_joint2_Q4 = C2_LA_derivatives_Q4[3:6]

C3_n_joint3_Q1 = C3_LA_derivatives_Q1[6:]
C3_n_joint3_Q2 = C3_LA_derivatives_Q2[6:]
C3_n_joint3_Q3 = C3_LA_derivatives_Q3[6:]
C3_n_joint3_Q4 = C3_LA_derivatives_Q4[6:]

C4_n_joint2_Q1 = C4_LA_derivatives_Q1[3:6]
C4_n_joint2_Q2 = C4_LA_derivatives_Q2[3:6]
C4_n_joint2_Q3 = C4_LA_derivatives_Q3[3:6]
C4_n_joint2_Q4 = C4_LA_derivatives_Q4[3:6]
C4_n_joint3_Q1 = C4_LA_derivatives_Q1[6:]
C4_n_joint3_Q2 = C4_LA_derivatives_Q2[6:]
C4_n_joint3_Q3 = C4_LA_derivatives_Q3[6:]
C4_n_joint3_Q4 = C4_LA_derivatives_Q4[6:]

In [303]:
joint1_center = Vec3(0)
joint2_center = Vec3(0)
joint3_center = Vec3(0)
joint4_center = Vec3(0)

# CABLE 1: acts on joint 1 only
joint1_center_in_C1_body_origin_Q1 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q1, joint1_center, C1_body_origin).to_numpy()
joint1_center_in_C1_body_origin_Q2 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q2, joint1_center, C1_body_origin).to_numpy()
joint1_center_in_C1_body_origin_Q3 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q3, joint1_center, C1_body_origin).to_numpy()
joint1_center_in_C1_body_origin_Q4 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q4, joint1_center, C1_body_origin).to_numpy()
joint1_center_in_C1_body_insertion_Q1 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q1, joint1_center, C1_body_insertion).to_numpy()
joint1_center_in_C1_body_insertion_Q2 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q2, joint1_center, C1_body_insertion).to_numpy()
joint1_center_in_C1_body_insertion_Q3 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q3, joint1_center, C1_body_insertion).to_numpy()
joint1_center_in_C1_body_insertion_Q4 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q4, joint1_center, C1_body_insertion).to_numpy()

joint1_normal_in_C1_body_origin_Q1 = joint1.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C1_n_joint1_Q1), C1_body_origin).to_numpy()
joint1_normal_in_C1_body_origin_Q2 = joint1.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C1_n_joint1_Q2), C1_body_origin).to_numpy()
joint1_normal_in_C1_body_origin_Q3 = joint1.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C1_n_joint1_Q3), C1_body_origin).to_numpy()
joint1_normal_in_C1_body_origin_Q4 = joint1.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C1_n_joint1_Q4), C1_body_origin).to_numpy()
joint1_normal_in_C1_body_insertion_Q1 = joint1.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C1_n_joint1_Q1), C1_body_insertion).to_numpy()
joint1_normal_in_C1_body_insertion_Q2 = joint1.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C1_n_joint1_Q2), C1_body_insertion).to_numpy()
joint1_normal_in_C1_body_insertion_Q3 = joint1.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C1_n_joint1_Q3), C1_body_insertion).to_numpy()
joint1_normal_in_C1_body_insertion_Q4 = joint1.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C1_n_joint1_Q4), C1_body_insertion).to_numpy()

# -> replacing the normals into origin and insertion bodies
C1_P1_body_origin = (to_vector(joint1_normal_in_C1_body_origin_Q1) + (to_vector(joint1_center_in_C1_body_origin_Q1) | to_vector(joint1_normal_in_C1_body_origin_Q1)) * ni).dual()
C1_P2_body_origin = (to_vector(joint1_normal_in_C1_body_origin_Q2) + (to_vector(joint1_center_in_C1_body_origin_Q2) | to_vector(joint1_normal_in_C1_body_origin_Q2)) * ni).dual()
C1_P3_body_origin = (to_vector(joint1_normal_in_C1_body_origin_Q3) + (to_vector(joint1_center_in_C1_body_origin_Q3) | to_vector(joint1_normal_in_C1_body_origin_Q3)) * ni).dual()
C1_P4_body_origin = (to_vector(joint1_normal_in_C1_body_origin_Q4) + (to_vector(joint1_center_in_C1_body_origin_Q4) | to_vector(joint1_normal_in_C1_body_origin_Q4)) * ni).dual()

C1_P1_body_insertion = (to_vector(joint1_normal_in_C1_body_insertion_Q1) + (to_vector(joint1_center_in_C1_body_insertion_Q1) | to_vector(joint1_normal_in_C1_body_insertion_Q1)) * ni).dual()
C1_P2_body_insertion = (to_vector(joint1_normal_in_C1_body_insertion_Q2) + (to_vector(joint1_center_in_C1_body_insertion_Q2) | to_vector(joint1_normal_in_C1_body_insertion_Q2)) * ni).dual()
C1_P3_body_insertion = (to_vector(joint1_normal_in_C1_body_insertion_Q3) + (to_vector(joint1_center_in_C1_body_insertion_Q3) | to_vector(joint1_normal_in_C1_body_insertion_Q3)) * ni).dual()
C1_P4_body_insertion = (to_vector(joint1_normal_in_C1_body_insertion_Q4) + (to_vector(joint1_center_in_C1_body_insertion_Q4) | to_vector(joint1_normal_in_C1_body_insertion_Q4)) * ni).dual()

C1_L_origin = C1_P1_body_origin.meet(C1_P2_body_origin).meet(C1_P3_body_origin).meet(C1_P4_body_origin)
C1_L_insertion = C1_P1_body_insertion.meet(C1_P2_body_insertion).meet(C1_P3_body_insertion).meet(C1_P4_body_insertion)
print(to_point(C1_O) ^ C1_L_origin)
print(to_point(C1_I) ^ C1_L_insertion)

# CABLE 2: acts on joint 1 and 2
joint1_center_in_C2_body_origin_Q1 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q1, joint1_center, C2_body_origin).to_numpy()
joint1_center_in_C2_body_origin_Q2 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q2, joint1_center, C2_body_origin).to_numpy()
joint1_center_in_C2_body_origin_Q3 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q3, joint1_center, C2_body_origin).to_numpy()
joint1_center_in_C2_body_origin_Q4 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q4, joint1_center, C2_body_origin).to_numpy()
joint1_center_in_C2_body_insertion_Q1 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q1, joint1_center, C2_body_insertion).to_numpy()
joint1_center_in_C2_body_insertion_Q2 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q2, joint1_center, C2_body_insertion).to_numpy()
joint1_center_in_C2_body_insertion_Q3 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q3, joint1_center, C2_body_insertion).to_numpy()
joint1_center_in_C2_body_insertion_Q4 = joint1.getParentFrame().findStationLocationInAnotherFrame(Q4, joint1_center, C2_body_insertion).to_numpy()

joint1_normal_in_C2_body_origin_Q1 = joint1.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C2_n_joint1_Q1), C2_body_origin).to_numpy()
joint1_normal_in_C2_body_origin_Q2 = joint1.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C2_n_joint1_Q2), C2_body_origin).to_numpy()
joint1_normal_in_C2_body_origin_Q3 = joint1.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C2_n_joint1_Q3), C2_body_origin).to_numpy()
joint1_normal_in_C2_body_origin_Q4 = joint1.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C2_n_joint1_Q4), C2_body_origin).to_numpy()
joint1_normal_in_C2_body_insertion_Q1 = joint1.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C2_n_joint1_Q1), C2_body_insertion).to_numpy()
joint1_normal_in_C2_body_insertion_Q2 = joint1.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C2_n_joint1_Q2), C2_body_insertion).to_numpy()
joint1_normal_in_C2_body_insertion_Q3 = joint1.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C2_n_joint1_Q3), C2_body_insertion).to_numpy()
joint1_normal_in_C2_body_insertion_Q4 = joint1.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C2_n_joint1_Q4), C2_body_insertion).to_numpy()
C2_P1_joint1_body_origin = (to_vector(joint1_normal_in_C2_body_origin_Q1) + (to_vector(joint1_center_in_C2_body_origin_Q1) | to_vector(joint1_normal_in_C2_body_origin_Q1)) * ni).dual()
C2_P2_joint1_body_origin = (to_vector(joint1_normal_in_C2_body_origin_Q2) + (to_vector(joint1_center_in_C2_body_origin_Q2) | to_vector(joint1_normal_in_C2_body_origin_Q2)) * ni).dual()
C2_P3_joint1_body_origin = (to_vector(joint1_normal_in_C2_body_origin_Q3) + (to_vector(joint1_center_in_C2_body_origin_Q3) | to_vector(joint1_normal_in_C2_body_origin_Q3)) * ni).dual()
C2_P4_joint1_body_origin = (to_vector(joint1_normal_in_C2_body_origin_Q4) + (to_vector(joint1_center_in_C2_body_origin_Q4) | to_vector(joint1_normal_in_C2_body_origin_Q4)) * ni).dual()
C2_P1_joint1_body_insertion = (to_vector(joint1_normal_in_C2_body_insertion_Q1) + (to_vector(joint1_center_in_C2_body_insertion_Q1) | to_vector(joint1_normal_in_C2_body_insertion_Q1)) * ni).dual()
C2_P2_joint1_body_insertion = (to_vector(joint1_normal_in_C2_body_insertion_Q2) + (to_vector(joint1_center_in_C2_body_insertion_Q2) | to_vector(joint1_normal_in_C2_body_insertion_Q2)) * ni).dual()
C2_P3_joint1_body_insertion = (to_vector(joint1_normal_in_C2_body_insertion_Q3) + (to_vector(joint1_center_in_C2_body_insertion_Q3) | to_vector(joint1_normal_in_C2_body_insertion_Q3)) * ni).dual()
C2_P4_joint1_body_insertion = (to_vector(joint1_normal_in_C2_body_insertion_Q4) + (to_vector(joint1_center_in_C2_body_insertion_Q4) | to_vector(joint1_normal_in_C2_body_insertion_Q4)) * ni).dual()


joint2_center_in_C2_body_origin_Q1 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q1, joint2_center, C2_body_origin).to_numpy()
joint2_center_in_C2_body_origin_Q2 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q2, joint2_center, C2_body_origin).to_numpy()
joint2_center_in_C2_body_origin_Q3 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q3, joint2_center, C2_body_origin).to_numpy()
joint2_center_in_C2_body_origin_Q4 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q4, joint2_center, C2_body_origin).to_numpy()
joint2_center_in_C2_body_insertion_Q1 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q1, joint2_center, C2_body_insertion).to_numpy()
joint2_center_in_C2_body_insertion_Q2 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q2, joint2_center, C2_body_insertion).to_numpy()
joint2_center_in_C2_body_insertion_Q3 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q3, joint2_center, C2_body_insertion).to_numpy()
joint2_center_in_C2_body_insertion_Q4 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q4, joint2_center, C2_body_insertion).to_numpy()

joint2_normal_in_C2_body_origin_Q1 = joint2.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C2_n_joint2_Q1), C2_body_origin).to_numpy()
joint2_normal_in_C2_body_origin_Q2 = joint2.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C2_n_joint2_Q2), C2_body_origin).to_numpy()
joint2_normal_in_C2_body_origin_Q3 = joint2.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C2_n_joint2_Q3), C2_body_origin).to_numpy()
joint2_normal_in_C2_body_origin_Q4 = joint2.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C2_n_joint2_Q4), C2_body_origin).to_numpy()
joint2_normal_in_C2_body_insertion_Q1 = joint2.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C2_n_joint2_Q1), C2_body_insertion).to_numpy()
joint2_normal_in_C2_body_insertion_Q2 = joint2.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C2_n_joint2_Q2), C2_body_insertion).to_numpy()
joint2_normal_in_C2_body_insertion_Q3 = joint2.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C2_n_joint2_Q3), C2_body_insertion).to_numpy()
joint2_normal_in_C2_body_insertion_Q4 = joint2.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C2_n_joint2_Q4), C2_body_insertion).to_numpy()
C2_P1_joint2_body_origin = (to_vector(joint2_normal_in_C2_body_origin_Q1) + (to_vector(joint2_center_in_C2_body_origin_Q1) | to_vector(joint2_normal_in_C2_body_origin_Q1)) * ni).dual()
C2_P2_joint2_body_origin = (to_vector(joint2_normal_in_C2_body_origin_Q2) + (to_vector(joint2_center_in_C2_body_origin_Q2) | to_vector(joint2_normal_in_C2_body_origin_Q2)) * ni).dual()
C2_P3_joint2_body_origin = (to_vector(joint2_normal_in_C2_body_origin_Q3) + (to_vector(joint2_center_in_C2_body_origin_Q3) | to_vector(joint2_normal_in_C2_body_origin_Q3)) * ni).dual()
C2_P4_joint2_body_origin = (to_vector(joint2_normal_in_C2_body_origin_Q4) + (to_vector(joint2_center_in_C2_body_origin_Q4) | to_vector(joint2_normal_in_C2_body_origin_Q4)) * ni).dual()
C2_P1_joint2_body_insertion = (to_vector(joint2_normal_in_C2_body_insertion_Q1) + (to_vector(joint2_center_in_C2_body_insertion_Q1) | to_vector(joint2_normal_in_C2_body_insertion_Q1)) * ni).dual()
C2_P2_joint2_body_insertion = (to_vector(joint2_normal_in_C2_body_insertion_Q2) + (to_vector(joint2_center_in_C2_body_insertion_Q2) | to_vector(joint2_normal_in_C2_body_insertion_Q2)) * ni).dual()
C2_P3_joint2_body_insertion = (to_vector(joint2_normal_in_C2_body_insertion_Q3) + (to_vector(joint2_center_in_C2_body_insertion_Q3) | to_vector(joint2_normal_in_C2_body_insertion_Q3)) * ni).dual()
C2_P4_joint2_body_insertion = (to_vector(joint2_normal_in_C2_body_insertion_Q4) + (to_vector(joint2_center_in_C2_body_insertion_Q4) | to_vector(joint2_normal_in_C2_body_insertion_Q4)) * ni).dual()

# ONLY TWO POSTURES ARE REQUIRED.
C2_L_joint1_origin = C2_P1_joint1_body_origin.meet(C2_P2_joint1_body_origin) #.meet(C2_P3_joint1_body_origin).meet(C2_P4_joint1_body_origin)
C2_L_joint1_insertion = C2_P1_joint1_body_insertion.meet(C2_P2_joint1_body_insertion) #.meet(C2_P3_joint1_body_insertion).meet(C2_P4_joint1_body_insertion)
C2_L_joint2_origin = C2_P1_joint2_body_origin.meet(C2_P2_joint2_body_origin) #.meet(C2_P3_joint2_body_origin).meet(C2_P4_joint2_body_origin)
C2_L_joint2_insertion = C2_P1_joint2_body_insertion.meet(C2_P2_joint2_body_insertion) #.meet(C2_P3_joint2_body_insertion).meet(C2_P4_joint2_body_insertion)
C2_L_origin = C2_L_joint1_origin.meet(C2_L_joint2_origin)
C2_L_insertion = C2_L_joint1_insertion.meet(C2_L_joint2_insertion)

print(to_point(C2_O) ^ C2_L_origin)
print(to_point(C2_I) ^ C2_L_insertion)

# IT's a point pair = it is a unique solution
sol_C2_O = C2_L_origin.normal()
print(sol_C2_O)
print(C2_O)

sol_C2_I = C2_L_insertion.normal()
print(sol_C2_I)
print(C2_I)

# print()
# print(C2_L_insertion.normal())

# # sol_C2_O = to_array(g3c_tools.intersect_line_and_plane_to_point(C2_L_joint1_origin, C2_P1_joint2_body_origin))
# # sol_C2_I = to_array(g3c_tools.intersect_line_and_plane_to_point(C2_L_joint1_insertion, C2_P2_joint2_body_insertion))
# print(sol_C2_O, C2_O)
# print(sol_C2_I, C2_I)
# # print(blade_type(C2_L_origin), blade_type(C2_L_insertion))

# CABLE 3: acts on joint 3 only
joint3_center_in_C3_body_origin_Q1 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q1, joint3_center, C3_body_origin).to_numpy()
joint3_center_in_C3_body_origin_Q2 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q2, joint3_center, C3_body_origin).to_numpy()
joint3_center_in_C3_body_origin_Q3 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q3, joint3_center, C3_body_origin).to_numpy()
joint3_center_in_C3_body_origin_Q4 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q4, joint3_center, C3_body_origin).to_numpy()
joint3_center_in_C3_body_insertion_Q1 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q1, joint3_center, C3_body_insertion).to_numpy()
joint3_center_in_C3_body_insertion_Q2 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q2, joint3_center, C3_body_insertion).to_numpy()
joint3_center_in_C3_body_insertion_Q3 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q3, joint3_center, C3_body_insertion).to_numpy()
joint3_center_in_C3_body_insertion_Q4 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q4, joint3_center, C3_body_insertion).to_numpy()

# directions of C3 in the joint frame
C3_D1 = C3_LA_derivatives_Q1[6:]
C3_D2 = C3_LA_derivatives_Q2[6:]
C3_D3 = C3_LA_derivatives_Q3[6:]
C3_D4 = C3_LA_derivatives_Q4[6:]

n1 = joint3.getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*C3_D1), C3_body_origin).to_numpy() @ np.array([e1, e2, e3])
n2 = joint3.getParentFrame().expressVectorInAnotherFrame(Q2, Vec3(*C3_D2), C3_body_origin).to_numpy() @ np.array([e1, e2, e3])
n3 = joint3.getParentFrame().expressVectorInAnotherFrame(Q3, Vec3(*C3_D3), C3_body_origin).to_numpy() @ np.array([e1, e2, e3])
n4 = joint3.getParentFrame().expressVectorInAnotherFrame(Q4, Vec3(*C3_D4), C3_body_origin).to_numpy() @ np.array([e1, e2, e3])

P1 = (n1 + (to_vector(joint3_center_in_C3_body_origin_Q1) | n1)*ni).dual()
P2 = (n2 + (to_vector(joint3_center_in_C3_body_origin_Q2) | n2)*ni).dual()
P3 = (n3 + (to_vector(joint3_center_in_C3_body_origin_Q3) | n3)*ni).dual()
P4 = (n4 + (to_vector(joint3_center_in_C3_body_origin_Q4) | n4)*ni).dual()
print(P1)
print(blade_type(P1))
L12 = P1.meet(P2)
L13 = P1.meet(P3)
L14 = P1.meet(P4)

I_in_O_Q1 = C3_body_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*C3_I), C3_body_origin).to_numpy()
LQ1 = to_point(C3_O) ^ to_point(I_in_O_Q1) ^ ni
# MQ1 = 
print(LQ1)
print(LQ1^P1)
print(blade_type(P1))
print(blade_type(P2))
print(blade_type(L12))
p, d = g3c_tools.line_to_point_and_direction(L12)
# print(p)
# print("p", to_vector(p))
# P12_ = (d + (to_vector(p) | d)*ni).dual()
# print(blade_type(P12_))
print(to_point(C3_O) ^ (L12 - (no^d^ni)))
# # so the moments are orthogonal to these normally
# D12 = g3c_tools.line_to_point_and_direction(L12)[1]
# D13 = g3c_tools.line_to_point_and_direction(L13)[1]
# D14 = g3c_tools.line_to_point_and_direction(L14)[1]
# print(D12)

# print(to_point(C3_O) ^ D12)
# print(to_point(C3_O) ^ D13)
# print(to_point(C3_O) ^ D14)
# # print(blade_type(P12))
# # print(blade_type(P13.dual()))
# # print(to_point(C3_O) ^ P12)

# CABLE 4: acts on joint 2 and 3
joint2_center_in_C4_body_origin_Q1 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q1, joint2_center, C4_body_origin).to_numpy()
joint2_center_in_C4_body_origin_Q2 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q2, joint2_center, C4_body_origin).to_numpy()
joint2_center_in_C4_body_origin_Q3 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q3, joint2_center, C4_body_origin).to_numpy()
joint2_center_in_C4_body_origin_Q4 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q4, joint2_center, C4_body_origin).to_numpy()
joint2_center_in_C4_body_insertion_Q1 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q1, joint2_center, C4_body_insertion).to_numpy()
joint2_center_in_C4_body_insertion_Q2 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q2, joint2_center, C4_body_insertion).to_numpy()
joint2_center_in_C4_body_insertion_Q3 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q3, joint2_center, C4_body_insertion).to_numpy()
joint2_center_in_C4_body_insertion_Q4 = joint2.getParentFrame().findStationLocationInAnotherFrame(Q4, joint2_center, C4_body_insertion).to_numpy()

joint3_center_in_C4_body_origin_Q1 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q1, joint3_center, C4_body_origin).to_numpy()
joint3_center_in_C4_body_origin_Q2 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q2, joint3_center, C4_body_origin).to_numpy()
joint3_center_in_C4_body_origin_Q3 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q3, joint3_center, C4_body_origin).to_numpy()
joint3_center_in_C4_body_origin_Q4 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q4, joint3_center, C4_body_origin).to_numpy()
joint3_center_in_C4_body_insertion_Q1 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q1, joint3_center, C4_body_insertion).to_numpy()
joint3_center_in_C4_body_insertion_Q2 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q2, joint3_center, C4_body_insertion).to_numpy()
joint3_center_in_C4_body_insertion_Q3 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q3, joint3_center, C4_body_insertion).to_numpy()
joint3_center_in_C4_body_insertion_Q4 = joint3.getParentFrame().findStationLocationInAnotherFrame(Q4, joint3_center, C4_body_insertion).to_numpy()


0
0
0
0
(0.05808^e14) + (0.05808^e15) + (0.86618^e24) + (0.86618^e25) + (0.60112^e34) + (0.60112^e35) - (1.0^e45)
[0.05808361 0.86617615 0.60111501]
(0.70807^e14) + (0.70807^e15) + (0.02058^e24) + (0.02058^e25) + (0.96991^e34) + (0.96991^e35) - (1.0^e45)
[0.70807258 0.02058449 0.96990985]
-(0.38385^e1234) - (0.38385^e1235) - (0.63401^e1245) + (0.76771^e1345) - (0.09307^e2345)
MultiVectorType.Plane
-(2.66796^e124) - (2.66796^e125) - (2.20073^e134) - (2.20073^e135) - (0.40096^e145) + (0.02138^e234) + (0.02138^e235) - (3.30725^e245) - (2.73128^e345)
0
MultiVectorType.Plane
MultiVectorType.Plane
MultiVectorType.Line
-(0.02125^e124) - (0.02125^e125) - (0.26066^e134) - (0.26066^e135) + (0.10006^e145) - (0.06185^e234) - (0.06185^e235) - (0.29127^e345)


In [284]:
k1 = g3c_tools.random_point_pair_at_origin()
k2 = g3c_tools.random_point_pair_at_origin()
print(k1)

k12 = k1.meet(k2)
print(k12)

-(4.49518^e14) - (4.50627^e15) - (13.42137^e24) - (13.45446^e25) + (1.47479^e34) + (1.47842^e35)
0.08741


In [255]:
print(R)
# print(np.log(R))
# Biv = R - R.scalar()
# print(R.basis_vector)
R.grades(0)
# R.grade(0)
# G3c.scalar(R)
print(R.as_array()[0])
cos_theta_2 = R.as_array()[0]
theta = 2 * np.arccos(cos_theta_2)
Biv = (R - R.as_array()[0]) / np.sin(theta)
print(Biv)
print(to_point(C3_O) ^ (Biv))

0.84224 + (0.00313^e12) - (0.39316^e13) + (0.15813^e23)
0.8422389782573404
(0.00344^e12) - (0.43294^e13) + (0.17413^e23)
(0.23751^e123) - (0.00039^e124) + (0.00305^e125) + (0.04955^e134) - (0.38339^e135) - (0.01993^e234) + (0.1542^e235)


In [23]:
plane_normal_Q1 = lever_arms_Q1[:3]
plane_location_Q1 = np.array([0,0,0])
plane_normal_Q2 = lever_arms_Q2[:3]
plane_location_Q2 = np.array([0,0,0])
plane_normal_Q3 = lever_arms_Q3[:3]
plane_location_Q3 = np.array([0,0,0])

# Let's replace the normal plane in each origin and insertion reference body.
# For this, the orientation of the normal is displaced, then the point of application is moved.
joint_frame = J[0].getParentFrame()

PO_normal_Q1 = joint_frame.expressVectorInAnotherFrame(Q1, Vec3(*plane_normal_Q1), B_origin)
PO_pos_Q1 = joint_frame.findStationLocationInAnotherFrame(Q1, Vec3(0), B_origin)

PO_normal_Q2 = joint_frame.expressVectorInAnotherFrame(Q2, Vec3(*plane_normal_Q2), B_origin)
PO_pos_Q2 = joint_frame.findStationLocationInAnotherFrame(Q2, Vec3(0), B_origin)

PO_normal_Q3 = joint_frame.expressVectorInAnotherFrame(Q3, Vec3(*plane_normal_Q3), B_origin)
PO_pos_Q3 = joint_frame.findStationLocationInAnotherFrame(Q3, Vec3(0), B_origin)

PI_normal_Q1 = joint_frame.expressVectorInAnotherFrame(Q1, Vec3(*plane_normal_Q1), B_insertion)
PI_pos_Q1 = joint_frame.findStationLocationInAnotherFrame(Q1, Vec3(0), B_insertion)
PI_normal_Q1 = Vec3(*(PI_normal_Q1.to_numpy() / np.linalg.norm(PI_normal_Q1.to_numpy())))

PI_normal_Q2 = joint_frame.expressVectorInAnotherFrame(Q2, Vec3(*plane_normal_Q2), B_insertion)
PI_normal_Q2 = Vec3(*(PI_normal_Q2.to_numpy() / np.linalg.norm(PI_normal_Q2.to_numpy())))
PI_pos_Q2 = joint_frame.findStationLocationInAnotherFrame(Q2, Vec3(0), B_insertion)

PI_normal_Q3 = joint_frame.expressVectorInAnotherFrame(Q3, Vec3(*plane_normal_Q3), B_insertion)
PI_normal_Q3 = Vec3(*(PI_normal_Q3.to_numpy() / np.linalg.norm(PI_normal_Q3.to_numpy())))
PI_pos_Q3 = joint_frame.findStationLocationInAnotherFrame(Q3, Vec3(0), B_insertion)

PO_Q1 = (to_vector(PO_normal_Q1) + (to_point(PO_pos_Q1) | to_vector(PO_normal_Q1)) * ni).dual()
PO_Q2 = (to_vector(PO_normal_Q2) + (to_point(PO_pos_Q2) | to_vector(PO_normal_Q2)) * ni).dual()
PO_Q3 = (to_vector(PO_normal_Q3) + (to_point(PO_pos_Q3) | to_vector(PO_normal_Q3)) * ni).dual()

PI_Q1 = (to_vector(PI_normal_Q1) + (to_point(PI_pos_Q1) | to_vector(PI_normal_Q1)) * ni).dual()
PI_Q2 = (to_vector(PI_normal_Q2) + (to_point(PI_pos_Q2) | to_vector(PI_normal_Q2)) * ni).dual()
PI_Q3 = (to_vector(PI_normal_Q3) + (to_point(PI_pos_Q3) | to_vector(PI_normal_Q3)) * ni).dual()
print(PO_Q1)

LO = PO_Q1.meet(PO_Q2)
LI = PI_Q1.meet(PI_Q2)

LO_2 = PO_Q2.meet(PO_Q3)
LI_2 = PI_Q2.meet(PI_Q3)

print(LO)
print(LI)
O = to_point(origin.to_numpy())
I = to_point(insertion.to_numpy())
print(O ^ LO)
print(I ^ LI)
print(O ^ LO_2)
print(I ^ LI_2)

print(LO.normal())
print(LO_2.normal())
print(LI.normal())
print(LI_2.normal())

# PO = LO.meet(LO_2)
# print(PO.normal())
# print(LO.normal())
PI = LI.meet(LI_2)
print(PI)

print(blade_type(PI))
# print(down(PI.dual()))



-(0.07345^e1234) - (0.07345^e1235) - (0.04239^e1245) - (0.25203^e1345) + (0.02355^e2345)
-(0.00443^e124) - (0.00443^e125) + (0.04463^e134) + (0.04463^e135) + (0.04098^e145) + (0.00467^e234) + (0.00467^e235) + (0.00127^e245) + (0.03034^e345)
(0.65658^e124) + (0.65658^e125) + (0.02276^e134) + (0.02276^e135) + (0.61317^e145) - (0.01555^e234) - (0.01555^e235) - (0.72089^e245) - (0.01047^e345)
0
0
0
0
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
(0.69373^e124) + (0.69373^e125) + (0.02405^e134) + (0.02405^e135) + (0.64786^e145) - (0.01642^e234) - (0.01642^e235) - (0.76168^e245) - (0.01106^e345)
-(0.51641^e124) - (0.51641^e125) + (0.04141^e134) + (0.04141^e135) - (0.40862^e145) + (0.25698^e234) + (0.25698^e235) + (0.87086^e245) - (0.27318

In [24]:
to_point(PI_pos_Q1) ^ PI_Q2

(0.63515^e12345)

In [25]:
joint_center_in_ground_Q1 = to_point(joint_frame.findStationLocationInGround(Q1, Vec3(0)))
joint_center_in_ground_Q2 = to_point(joint_frame.findStationLocationInGround(Q1, Vec3(0)))
normal_in_ground_Q1 = to_vector(joint_frame.expressVectorInGround(Q1, Vec3(*plane_normal_Q1)))
normal_in_ground_Q2 = to_vector(joint_frame.expressVectorInGround(Q2, Vec3(*plane_normal_Q2)))

PG_Q1 = (normal_in_ground_Q1 + (joint_center_in_ground_Q1 | normal_in_ground_Q1) * ni).dual()
PG_Q2 = (normal_in_ground_Q2 + (joint_center_in_ground_Q2 | normal_in_ground_Q2) * ni).dual()
L = PG_Q1.meet(PG_Q2)
O_in_ground_Q1 = to_point(B_origin.findStationLocationInGround(Q1, origin))
O_in_ground_Q2 = to_point(B_origin.findStationLocationInGround(Q2, origin))
I_in_ground_Q1 = to_point(B_insertion.findStationLocationInGround(Q1, insertion))
I_in_ground_Q2 = to_point(B_insertion.findStationLocationInGround(Q2, insertion))
print(O_in_ground_Q1 ^ L)
print(I_in_ground_Q1 ^ L)
print(I_in_ground_Q2 ^ L)


0
-(0.03491^e1234) - (0.03491^e1235) - (0.02014^e1245) - (0.11978^e1345) + (0.01119^e2345)
-(0.05401^e1234) - (0.05401^e1235) - (0.04981^e1245) + (0.00228^e1345) + (0.03695^e2345)


In [26]:
PJ_Q1 = to_vector(plane_normal_Q1).dual()
PJ_Q2 = to_vector(plane_normal_Q2).dual()
print(PJ_Q1)
print(PJ_Q2)

LJ = PJ_Q1.meet(PJ_Q2)
print("LJ", LJ)



# pos, D = to_array(LO)
# print(pos, D)

# dirt1 = B_origin.expressVectorInAnotherFrame(Q1, Vec3(*D), joint_frame)
# # L_test = to
# print(dirt1)
# LO_test = no ^ to_vector(dirt1) ^ ni
# print(LO_test.normal())
# print(LO_test ^ LJ)

O_in_joint_Q1 = to_point(B_origin.findStationLocationInAnotherFrame(Q1, origin, joint_frame))
O_in_joint_Q2 = to_point(B_origin.findStationLocationInAnotherFrame(Q2, origin, joint_frame))
I_in_joint_Q1 = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, insertion, joint_frame))
I_in_joint_Q2 = to_point(B_insertion.findStationLocationInAnotherFrame(Q2, insertion, joint_frame))
print(O_in_joint_Q1^LJ)
print(O_in_joint_Q2^LJ)
print(I_in_joint_Q1^LJ)
print(I_in_joint_Q2^LJ)

(0.00017^e1245) - (0.23478^e1345) - (0.10368^e2345)
(0.14348^e1245) - (0.08325^e1345) + (0.11355^e2345)
LJ -(0.03367^e145) - (0.01489^e245) + (0.03529^e345)
0
0
(8e-05^e1245) - (0.11158^e1345) - (0.04927^e2345)
(0.0443^e1245) - (0.0257^e1345) + (0.03506^e2345)


In [27]:
to_point(PO_pos_Q1) ^ LO

0

In [28]:
ml = PI_Q1.meet(PI_Q2.meet(PI_Q3))
print(I)
print(ml)
print(ml.normal())
mlO = PO_Q1.meet(PO_Q2.meet(PO_Q3))
print(mlO.normal())
print(O)
print(I ^ PI_Q1)
print(I ^ PI_Q2)
print(I ^ PI_Q3)
print(blade_type(I))
Lk = PI_Q1.meet(PI_Q2)
print(blade_type(Lk))
Lk2 = g3c_tools.intersect_line_and_plane_to_point(PI_Q1.meet(PI_Q2), PI_Q3)
print(Lk2)
print(I)

-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)
-(0.0714^e14) - (0.0714^e15) - (0.29463^e24) - (0.29463^e25) - (0.0119^e34) - (0.0119^e35) - (0.35355^e45)
-(0.20196^e14) - (0.20196^e15) - (0.83336^e24) - (0.83336^e25) - (0.03367^e34) - (0.03367^e35) - (1.0^e45)
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
(0.20977^e1) + (0.11476^e2) - (0.93389^e3) - (0.03534^e4) + (0.96466^e5)
0
0
0
MultiVectorType.ConformalPoint
MultiVectorType.Line
-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)
-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)


In [29]:
pI_interest = no ^ down(Lk2) ^ ni
pI_interest

-(0.20196^e145) - (0.83336^e245) - (0.03367^e345)

In [37]:
O_random = g3c_tools.random_conformal_point()
O_random = g3c_tools.project_points_to_line([O_random], LO)[0]
I_random = g3c_tools.random_conformal_point()
# I_random = g3c_tools.project_points_to_line([I_random], PI)[0]
I_random = (I_random | PI) / PI
# I_random = (I_random | pI_interest) / pI_interest

lever_arms_Q1_random = lever_arms(model, Q1, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q2_random = lever_arms(model, Q2, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q3_random = lever_arms(model, Q3, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q4_random = lever_arms(model, Q4, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q5_random = lever_arms(model, Q5, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q6_random = lever_arms(model, Q6, W, O_random, B_origin, I_random, B_insertion)

print(to_array(O))
print(to_array(I_random))


print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((lever_arms_Q1_random[:3] / np.linalg.norm(lever_arms_Q1_random[:3])).round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((lever_arms_Q2_random[:3] / np.linalg.norm(lever_arms_Q2_random[:3])).round(4).tolist())

print((lever_arms_Q3[:3] / np.linalg.norm(lever_arms_Q3[:3])).round(4).tolist())
print((lever_arms_Q3_random[:3] / np.linalg.norm(lever_arms_Q3_random[:3])).round(4).tolist())

print((lever_arms_Q4[:3] / np.linalg.norm(lever_arms_Q4[:3])).round(4).tolist())
print((lever_arms_Q4_random[:3] / np.linalg.norm(lever_arms_Q4_random[:3])).round(4).tolist())

print((lever_arms_Q5[:3] / np.linalg.norm(lever_arms_Q5[:3])).round(4).tolist())
print((lever_arms_Q5_random[:3] / np.linalg.norm(lever_arms_Q5_random[:3])).round(4).tolist())

print((lever_arms_Q6[:3] / np.linalg.norm(lever_arms_Q6[:3])).round(4).tolist())
print((lever_arms_Q6_random[:3] / np.linalg.norm(lever_arms_Q6_random[:3])).round(4).tolist())

[ 0.20976702  0.11475728 -0.93389042]
[-0.20195964 -0.83335775 -0.03366729]
[0.404, -0.9148, -0.0006]
[0.404, -0.9148, -0.0006]
[-0.5649, -0.4141, -0.7137]
[-0.5649, -0.4141, -0.7137]
[-0.4995, 0.8588, -0.1141]
[-0.4995, 0.8588, -0.1141]
[-0.5744, 0.7899, -0.2146]
[-0.5744, 0.7899, -0.2146]
[-0.751, 0.2479, -0.6119]
[-0.751, 0.2479, -0.6119]
[0.5578, -0.8077, 0.1913]
[0.5578, -0.8077, 0.1913]


In [51]:
pointI = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random)), ground))
pointO = to_point(B_origin.findStationLocationInAnotherFrame(Q1, origin, ground))
LOI = pointO^pointI^ni
I_random2 = g3c_tools.random_conformal_point()
I_random2 = (I_random2 | LOI) / LOI
print(I_random2 ^ LOI)
pointI2 = to_point(ground.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random2)), B_insertion))

I_random_in_ground = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random)), ground))
print(O^LOI)
print(I_random_in_ground^LOI)
print(I_random2^LOI)

lever_arms_Q1_random = lever_arms(model, Q1, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q2_random = lever_arms(model, Q2, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q3_random = lever_arms(model, Q3, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q4_random = lever_arms(model, Q4, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q5_random = lever_arms(model, Q5, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q6_random = lever_arms(model, Q6, W, O_random, B_origin, pointI2, B_insertion)

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((lever_arms_Q1_random[:3] / np.linalg.norm(lever_arms_Q1_random[:3])).round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((lever_arms_Q2_random[:3] / np.linalg.norm(lever_arms_Q2_random[:3])).round(4).tolist())

print((lever_arms_Q3[:3] / np.linalg.norm(lever_arms_Q3[:3])).round(4).tolist())
print((lever_arms_Q3_random[:3] / np.linalg.norm(lever_arms_Q3_random[:3])).round(4).tolist())

print((lever_arms_Q4[:3] / np.linalg.norm(lever_arms_Q4[:3])).round(4).tolist())
print((lever_arms_Q4_random[:3] / np.linalg.norm(lever_arms_Q4_random[:3])).round(4).tolist())

print((lever_arms_Q5[:3] / np.linalg.norm(lever_arms_Q5[:3])).round(4).tolist())
print((lever_arms_Q5_random[:3] / np.linalg.norm(lever_arms_Q5_random[:3])).round(4).tolist())

print((lever_arms_Q6[:3] / np.linalg.norm(lever_arms_Q6[:3])).round(4).tolist())
print((lever_arms_Q6_random[:3] / np.linalg.norm(lever_arms_Q6_random[:3])).round(4).tolist())

0
0
0
0
[0.404, -0.9148, -0.0006]
[0.404, -0.9148, -0.0006]
[-0.5649, -0.4141, -0.7137]
[-0.4918, -0.5282, -0.6922]
[-0.4995, 0.8588, -0.1141]
[-0.6489, 0.6857, -0.3297]
[-0.5744, 0.7899, -0.2146]
[-0.6106, -0.3278, -0.7209]
[-0.751, 0.2479, -0.6119]
[-0.6106, 0.7452, -0.268]
[0.5578, -0.8077, 0.1913]
[0.3071, -0.9457, -0.1061]


In [58]:
I_random^down(I_random2)^ni
# I_random2

-(1.0^e45)

In [57]:
no^down(I_random2)^ni

-(0.56824^e145) + (3.31411^e245) + (17.6572^e345)

In [274]:
print(PI.normal())
print(((PI.normal()|(no))^ni)|no)

-(0.7911^e14) - (0.7911^e15) - (0.47482^e24) - (0.47482^e25) - (0.7111^e34) - (0.7111^e35) + (1.0^e45)
-(0.7911^e1) - (0.47482^e2) - (0.7111^e3) + (0.5^e4) - (0.5^e5)


In [275]:

print(lever_arms_by_derivatives_Q1.ravel().round(4).tolist())
print(lever_arms_by_derivatives_Q2.ravel().round(4).tolist())
lever_arms_Q1 = []
lever_arms_Q2 = []
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, O_random, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, I_random, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, O_random, FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, I_random, FJ).to_numpy()

    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)

# print(lever_arms_Q1.round(4).tolist())
print(lever_arms_Q1[:3].round(4).tolist() / np.linalg.norm(lever_arms_Q1[:3].round(4).tolist()))
print(lever_arms_by_derivatives_Q1.ravel()[:3].round(4).tolist() / np.linalg.norm(lever_arms_by_derivatives_Q1.ravel()[:3]))

# print(lever_arms_Q2.round(4).tolist())
# print(lever_arms_by_derivatives_Q2.ravel().round(4).tolist())
print(lever_arms_Q2[:3].round(4).tolist() / np.linalg.norm(lever_arms_Q2[:3].round(4).tolist()))
print(lever_arms_by_derivatives_Q2.ravel()[:3].round(4).tolist() / np.linalg.norm(lever_arms_by_derivatives_Q2.ravel()[:3]))

[0.7256, -0.7664, 0.9272, 1.1941, 1.1508, 0.2053, -0.675, -0.5937, -0.2545, -0.0895, -0.3261, 0.3516]
[0.6668, -0.5551, 0.3662, 0.252, -1.1602, -0.2614, 0.6664, 0.6613, 0.8102, -0.1866, -0.2008, 0.6752]


TypeError: in method 'Frame_findStationLocationInAnotherFrame', argument 3 of type 'SimTK::Vec3 const &'

In [12]:

# n_I_q1 = J[0].getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*n1), B_origin).to_numpy()
# Otest = J[0].getParentFrame().findStationLocationInAnotherFrame(Q1, Vec3(0), B_origin).to_numpy()

print(S ^ up(to_multivec(O_in_ground + 0.1*L1_dir_in_ground)))
print(S ^ up(to_multivec(I_in_ground + 0.1*L2_dir_in_ground)))
print(S ^ up(to_multivec(I_in_ground - 50*L2_dir_in_ground)))

k1 = (g3c_tools.random_conformal_point() | S) / S
k2 = (g3c_tools.random_conformal_point() | S) / S
print(k1)
print(S ^ k1)
print(S ^ k2)
kO = point_to_np(k1)
kI = point_to_np(k2)
kO_in = Vec3(*ground.findStationLocationInAnotherFrame(Q1, Vec3(*kO), B_origin).to_numpy())
kI_in = Vec3(*ground.findStationLocationInAnotherFrame(Q1, Vec3(*kI), B_insertion).to_numpy())

kO2_in = Vec3(*ground.findStationLocationInAnotherFrame(Q2, Vec3(*kO), B_origin).to_numpy())
kI2_in = Vec3(*ground.findStationLocationInAnotherFrame(Q2, Vec3(*kI), B_insertion).to_numpy())

lever_arms_Q1 = []
lever_arms_Q2 = []
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, kO_in, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, kI_in, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, Vec3(*kO), FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, Vec3(*kI), FJ).to_numpy()
    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)


la_by_derivatives_Q1 = common.lever_arm_matrix(Q1, [muscle], model.getCoordinateSet())
la_by_derivatives_Q2 = common.lever_arm_matrix(Q2, [muscle], model.getCoordinateSet())

print(lever_arms_Q1.round(4).tolist())
print(la_by_derivatives_Q1.ravel().round(4).tolist())

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((la_by_derivatives_Q1[:3] / np.linalg.norm(la_by_derivatives_Q1[:3])).ravel().round(4).tolist())

print("OK")
print(lever_arms_Q2.round(4).tolist())
print(la_by_derivatives_Q2.ravel().round(4).tolist())
print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((la_by_derivatives_Q2[:3] / np.linalg.norm(la_by_derivatives_Q2[:3])).ravel().round(4).tolist())

0
0
0
-(1.35764^e1) + (7.06332^e2) + (4.74274^e3) + (58.29926^e4) + (59.29926^e5)
0
0
[1.0354, -0.8731, -3.0674, -2.5531, 1.3454, -0.7172, -1.0005, 3.1802, -1.954, 3.6992, -1.145, -1.3789]
[0.0706, -0.0596, -0.2092, -0.0854, 0.3853, 1.3011, -0.2036, -0.0042, -0.7931, 0.68, -0.3989, -0.1945]
[0.3088, -0.2604, -0.9148]
[0.3088, -0.2604, -0.9148]
OK
[-0.5595, -0.0589, 1.2261, -0.6369, -0.0773, 0.5306, -0.0176, 0.8053, -0.9171, 1.2382, -1.8551, 0.4389]
[-0.1876, -0.0005, -0.1596, 0.0515, -0.6161, -0.7833, 0.1244, 0.4488, 1.3303, -0.6104, 0.6434, 1.0534]
[-0.4147, -0.0437, 0.9089]
[-0.7617, -0.0022, -0.648]


In [154]:
%matplotlib qt5

import matplotlib.pyplot as plt
import itertools

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")

all_O_test = []
all_I_test = []
alphas = np.linspace(-0.1,0.1,20)
for a in alphas:
    O_test = (origin.to_numpy() + a*direction_vec)
    O_test = B_origin.expressVectorInGround(Q1, Vec3(*O_test)).to_numpy()
    all_O_test.append(O_test)

    I_test = (insertion.to_numpy() + a*direction_vec2)
    I_test = B_insertion.expressVectorInGround(Q1, Vec3(*I_test)).to_numpy()
    all_I_test.append(I_test)
    
permut = itertools.permutations(list(range(len(alphas))), 2)
for permu in permut:
    i1, i2 = permu
    ax.plot([all_O_test[i1][0], all_I_test[i2][0]],[all_O_test[i1][1], all_I_test[i2][1]],[all_O_test[i1][2], all_I_test[i2][2]])#, c="red" )

# ax.plot([O_in_ground[0], I_in_ground[0]],[O_in_ground[1], I_in_ground[1]],[O_in_ground[2], I_in_ground[2]], c="red" )
# ax.plot([O2_in_ground[0], I2_in_ground[0]],[O2_in_ground[1], I2_in_ground[1]],[O2_in_ground[2], I2_in_ground[2]], c="blue" )

ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_zlim(-2,2)

(-2.0, 2.0)

In [9]:
lever_arms_Q1 = []
lever_arms_Q2 = []
new_origin_vec = Vec3(*new_origin_test)
new_insertion_vec = Vec3(*new_insertion_test)
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, new_origin_vec, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, new_insertion_vec, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, new_origin_vec, FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, new_insertion_vec, FJ).to_numpy()

    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)


la_by_derivatives_Q1 = common.lever_arm_matrix(Q1, [muscle], model.getCoordinateSet())
la_by_derivatives_Q2 = common.lever_arm_matrix(Q2, [muscle], model.getCoordinateSet())

print(lever_arms_Q1.round(4).tolist())
print(la_by_derivatives_Q1.ravel().round(4).tolist())

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((la_by_derivatives_Q1[:3] / np.linalg.norm(la_by_derivatives_Q1[:3])).ravel().round(4).tolist())

print(lever_arms_Q2.round(4).tolist())
print(la_by_derivatives_Q2.ravel().round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((la_by_derivatives_Q2[:3] / np.linalg.norm(la_by_derivatives_Q2[:3])).ravel().round(4).tolist())


[0.0722, -0.0608, -0.2138, -0.0893, 0.3872, 1.2977, -0.2051, 0.0012, -0.7951, 0.685, -0.4002, -0.1965]
[0.0706, -0.0596, -0.2092, -0.0854, 0.3853, 1.3011, -0.2036, -0.0042, -0.7931, 0.68, -0.3989, -0.1945]
[0.3088, -0.2604, -0.9148]
[0.3088, -0.2604, -0.9148]
[-0.1916, -0.0005, -0.163, 0.0535, -0.6136, -0.7816, 0.1209, 0.446, 1.332, -0.6102, 0.6466, 1.0553]
[-0.1876, -0.0005, -0.1596, 0.0515, -0.6161, -0.7833, 0.1244, 0.4488, 1.3303, -0.6104, 0.6434, 1.0534]
[-0.7617, -0.0022, -0.648]
[-0.7617, -0.0022, -0.648]


In [118]:
# LA = g3c_tools.random_conformal_point()^g3c_tools.random_conformal_point()^ni
# LA

-(84.0107^e124) - (84.0107^e125) - (18.00065^e134) - (18.00065^e135) + (7.5526^e145) - (68.79159^e234) - (68.79159^e235) - (27.31799^e245) - (12.03772^e345)

In [555]:
def null_rotor():
    return (0.0 * Px).exp()

def euler_extrinsic(rx, ry, rz):
    return (-0.5 * rx * Px).exp() * (-0.5 * ry * Py).exp() * (-0.5 * rz * Pz).exp()

def euler_intrinsic(rx, ry, rz):
    return  (-0.5 * rz * Pz).exp() * (-0.5 * ry * Py).exp() * (-(rx*Px) / 2).exp()

def translator(tx, ty, tz):
    return (-(tx*(e1^ni) + ty*(e2^ni) + tz*(e3^ni))/2).exp()

X = up(3*e1 + e2 + e3)

toG = null_rotor()
toB1 = null_rotor()
toJ1 = translator(*t1) * euler_extrinsic(*r1)
toJ1_child = euler_extrinsic(*q1)
toB2 = (translator(*t2) * euler_extrinsic(*r2)).inv()
toJ2 = translator(*t3) * euler_extrinsic(*r3)
toJ2_child = euler_extrinsic(*q2)
toB3 = (translator(*t4) * euler_extrinsic(*r4)).inv()

R = toG * toB1 * toJ1 * toJ1_child * toB2 * toJ2 * toJ2_child * toB3
print(down(R * X * ~R))

R = ~(toJ1_child * toB2)
print(down(R * X * ~R))



-(3.65384^e1) - (1.47323^e2) - (3.80405^e3)
-(3.53275^e1) - (0.62826^e2) + (0.63153^e3)


In [556]:
X = [1,2,3]
J_osim_dir = common.station_jacobian(model, state, X, B3)
print(J_osim_dir)
J_osim_moment = common.frame_jacobian(model, state, B3)
print(J_osim_moment)

[[ 4.81445088  0.38097219 -5.40392184 -3.91008054  0.19380367 -1.61455302]
 [-2.55270241 -1.60491412  0.52423453  1.72284215  1.14322941 -2.32923028]
 [ 0.59081906  1.33186891  1.48128821 -0.63139483 -1.23297924  2.86814419]]
[[-0.450548   -0.11785585 -0.09717794  0.34988903  0.56250767 -0.75490528]
 [-0.73119138  0.61744313  0.7722644   0.50490582  0.56033015  0.64796912]
 [ 0.51221642  0.77773645 -0.62782493 -0.78908034  0.60795991  0.10126223]]


In [572]:
EE = up(e1 + 2*e2 + 3*e3)
o = no # center of J1

Base = np.array([e1, e2, e3])

# Describing the origin in J1
R = ~(toG * toB1 * toJ1)
o_in_J1 = R * no * ~R
# o_in_J1 = no
# o_in_J1 = up(e1)
print(point_to_np(o_in_J1))
print(ground.findStationLocationInAnotherFrame(state, Vec3(0), J1.getParentFrame()).to_numpy())
print()
EE_in_origin = (B3.findStationLocationInGround(state, Vec3(1,2,3)).to_numpy())


R =  toJ1_child * toB2 * toJ2 * toJ2_child * toB3
EE_in_J1 = R * EE * ~R
print(down(EE_in_J1))
print(B3.findStationLocationInAnotherFrame(state, Vec3(1,2,3), J1.getParentFrame()).to_numpy())

# PLA = EE_in_J1^(e1 - down(EE_in_J1))^ni
PLA = (no^down(EE_in_J1)^(e1)^ni) # Plane centered at no works ?
PLA = (down(EE_in_J1)^(e1)^ni) # Direction works
PLA = (o_in_J1^down(EE_in_J1)^(e1)^ni) # Plane centered at joint works
# PLA = o_in_J1 ^ EE_in_J1 ^ up(e1)
PLA = (o_in_J1^e1^ni) + (down(EE_in_J1) ^ e1 ^ ni)
PLA = (o_in_J1 + down(EE_in_J1)) ^ e1 ^ ni # Line = sphere^dir^ni
PLA = up(down(o_in_J1) + down(EE_in_J1)) ^ e1 ^ ni # Line = sphere^dir^ni
# PLA = (EE_in_J1 ^ o_in_J1 ^ e1) ^ ni
R = toG * toB1 * toJ1
print()
print("PLA", PLA)
PLA_in_ground = R * PLA * ~R
print("PLA in G", PLA_in_ground.dual())
print("PLA in G", PLA_in_ground)
print("PLA in G", PLA_in_ground.normal())


[0.8617347  0.0625609  0.17393678]
[0.8617347  0.0625609  0.17393678]

(1.28791^e1) + (2.7502^e2) - (4.74138^e3)
[ 1.28790986  2.75020029 -4.74138085]

PLA -(2.81276^e124) - (2.81276^e125) + (4.56744^e134) + (4.56744^e135) + (1.0^e145)
PLA in G -(0.51222^e12) - (0.73119^e13) + (4.81445^e14) + (4.81445^e15) + (0.45055^e23) - (2.5527^e24) - (2.5527^e25) + (0.59082^e34) + (0.59082^e35)
PLA in G -(0.59082^e124) - (0.59082^e125) - (2.5527^e134) - (2.5527^e135) - (0.45055^e145) - (4.81445^e234) - (4.81445^e235) - (0.73119^e245) + (0.51222^e345)
PLA in G -(0.59082^e124) - (0.59082^e125) - (2.5527^e134) - (2.5527^e135) - (0.45055^e145) - (4.81445^e234) - (4.81445^e235) - (0.73119^e245) + (0.51222^e345)


In [541]:
OK = (o_in_J1 + down(EE_in_J1)) ^ e1
K = R * OK * ~R
print(K)
print((K^ni)^(R*e1*~R))

(2.14078^e12) - (0.70375^e13) + (0.46193^e14) + (0.30135^e15) - (4.07145^e23) - (1.58795^e24) - (2.57479^e25) + (1.40053^e34) + (1.41954^e35) - (0.33205^e45)


In [530]:
o_in_J1 + down(EE_in_J1)

(0.01505^e1) - (4.1251^e2) + (1.57185^e3) + (0.49072^e4) + (1.49072^e5)

In [536]:
print(down(EE_in_J1))
print(down(o_in_J1))
down(o_in_J1) + down(EE_in_J1)

(0.3471^e1) - (3.66956^e2) + (2.86169^e3)
-(0.33205^e1) - (0.45553^e2) - (1.28984^e3)


(0.01505^e1) - (4.1251^e2) + (1.57185^e3)

In [413]:
o_in_J1 = ground.findStationLocationInAnotherFrame(state, Vec3(0), J1.getParentFrame()).to_numpy()
EE_in_J1 = B3.findStationLocationInAnotherFrame(state, Vec3(1,2,3), J1.getParentFrame()).to_numpy()

Base = np.array([e1, e2, e3])
L = up(o_in_J1@Base) ^ up(EE_in_J1@Base) ^ ni
print(L)

omega = np.cross(o_in_J1, EE_in_J1)
print(omega)

-(5.9099^e124) - (5.9099^e125) + (2.96217^e134) + (2.96217^e135) + (2.9805^e145) + (2.04595^e234) + (2.04595^e235) - (3.01239^e245) + (2.5417^e345)
[ 2.0459544  -2.96216906 -5.90990259]
